In [ ]:
# TPA End
def send_init_TPA_metadata(root, file_hashes):
  # Storing Metadata to DB
  version_roots_TPA.append(root)
  file_version_hashes_TPA.append(file_hashes)

  n = len(file_hashes)
  sibling_nodes = []

  challenge_index = 0
  get_sibling_nodes(node = root, low = 0, high = n-1, pos = challenge_index, sibling_nodes = sibling_nodes)

  print("Sibs:", sibling_nodes)
  # root_proof = calculate_root_from_sib_path(input_hash = file_hashes[challenge_index], sibling_nodes = sibling_nodes)

  print(root.data)
  print(root_proof)
  if root.data == root_proof:
    return True
  else:
    return False

send_init_TPA_metadata(version_roots_client[0], file_hashes_temp)


In [ ]:
# Server End
def initial_upload_by_user(encrypted_blocks):
  n = len(encrypted_blocks)
  v0_root_server = buildMerkleTree(encrypted_blocks, Node(0), 0, n-1)
  version_roots_server.append(v0_root_server)
  return v0_root_server.data

# def retrive_file(version_no):

def challenge_server(version_no, low, high, input_hash, pos, sibling_nodes):
  print(low, high, pos)
  get_sibling_nodes(version_roots_server[version_no], low, high, pos, sibling_nodes)


# def verify_server_response(input_hash, sibling_nodes):
  

sib_nodes = []
challenge_idx = 0

n = len(file_version_hashes_TPA[0])
input_hash = calcBlockHash(file_version_hashes_TPA[0][challenge_idx])
challenge_server(0, 0, n-1, input_hash, 1, sib_nodes)

print(sib_nodes)

# print(encrypted_file_blocks_client[0][1].encode('utf-8'))
# print(file_version_hashes_client[0])

# encrypted_file_blocks_client[0][challenge_idx].encode('utf-8')


proof = calculate_root_from_sib_path(calcBlockHash(encrypted_file_blocks_client[0][1].encode('utf-8')), sib_nodes) ##################

print(proof)
print(version_roots_client[0].data)
print(version_roots_server[0].data)
print(version_roots_TPA[0].data)

In [ ]:
## CLIENT END 

file_version_hashes_client = []
# encrypted_file_blocks_client = []

def generate_encrypted_file_blocks(text_file):
  encrypted_file_blocks = []
  raw_file_blocks = padding_64(text_file) 
  for i in raw_file_blocks:
    encrypted_file_blocks.append(encrypt_block(i))
    # print(encrypted_file_blocks[-1])
  return encrypted_file_blocks 

def init_client():
  # Encrypting File Blocks and Generated Hashes
  global encrypted_file_blocks_client, file_version_hashes_client
  clear_all_db() ## Deleting all Stored Information

  encrypted_file_blocks_client.append(generate_encrypted_file_blocks(text_file))
  file_version_hashes_client.append(calcFileBlockHashes(encrypted_file_blocks_client[0]))

  # Creating First Merkle Tree Version
  n = len(encrypted_file_blocks_client[0])
  v0_root_client = buildMerkleTree(encrypted_file_blocks_client[0], Node(0), 0, n-1)
  version_roots_client.append(v0_root_client)

  # Uploading File to Server
  returned_merkle_root = initial_upload_by_user(encrypted_file_blocks_client[0])
  if(v0_root_client.data == returned_merkle_root):
    print("## File is Successfully uploaded to server for the first time")
  else:
    print("## File uploading ERROR!! ##")

  # Sending Metadata to TPA
  if(send_init_TPA_metadata(v0_root_client, file_version_hashes_client[0])):
    print("## Metadata is updated at TPA ##")
  else:
    print("## Metadata Update ERROR!! ##")


init_client()
print(version_roots_client)
print(file_version_hashes_client[0])

In [ ]:
# Client Side DB
version_roots_client = []
encrypted_file_blocks_client = []


In [ ]:
def clear_client_db():
  global version_roots_client, encrypted_file_blocks_client
  version_roots_client = []
  encrypted_file_blocks_client = []
clear_client_db()

In [ ]:
# TPA Side_DB
version_roots_TPA = []
file_version_hashes_TPA = [] # 2D list. Hash list for each updated version is appended

In [ ]:
def clear_TPA_db():
  global version_roots_TPA, file_version_hashes_TPA
  version_roots_TPA = []
  file_version_hashes_TPA = []
clear_TPA_db()

In [ ]:
# Server Side DB
version_roots_server = []

In [ ]:
def clear_server_db():
  global version_roots_server
  version_roots_server = []
clear_server_db()

In [ ]:
def clear_all_db():
  clear_client_db()
  clear_TPA_db()
  clear_server_db()

clear_all_db()

In [ ]:
print(version_roots_client)
print(version_roots_server)
print(version_roots_TPA)

In [29]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    print(obj)
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        print("*dhuke ", obj.values())
        print("*dhuke ", obj.keys())
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        print("dhuke 1 ", obj.__dict__)
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        print("dhuke 2")
        size += sum([get_size(i, seen) for i in obj])
    return size

In [30]:
import sys 
class Test:
  def __init__(self):
    self.number = 1000000000000000000
    self.string1 = "ab"
    self.string2 = "ab"
    self.left = None
    pass


t = Test()
print(sys.getsizeof(t), t.__sizeof__(), get_size(t))
#print(sys.getsizeof(t.number),sys.getsizeof(t.string))
#print(t.number.__sizeof__(),t.string.__sizeof__())
_dict = {}
_dict[t] = True


dhuke 1  {'number': 1000000000000000000, 'string1': 'ab', 'string2': 'ab', 'left': None}
{'number': 1000000000000000000, 'string1': 'ab', 'string2': 'ab', 'left': None}
*dhuke  dict_values([1000000000000000000, 'ab', 'ab', None])
*dhuke  dict_keys(['number', 'string1', 'string2', 'left'])
1000000000000000000
ab
ab
None
number
string1
string2
left
64 32 503
